# Churn Predictor
-----------------------------------

Kode untuk melakukan prediksi menggunakan model yang sudah di training sebelumnya.
Untuk menggunakan data baru, silahkan sesuaikan format data menggunakan xlsx, dan fitur yang sama persis
seperti data yang digunakan pada proses training.
--------------------------------
Tim Data Something
 DSW 2023

## Library dan Input File

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
# Masukan nama file dari dataframe ke sini
filename = "Telco_customer_churn_adapted_v2.xlsx"

## Encode dan Decode
Fungsi ini digunakan untuk meningkatkan performa pada prediksi

In [6]:
def decode(df):
    dfret = df
    # Reverse Payment Method
    df1 = dfret[["Debit", "Credit", "Pulsa", "Digital Wallet"]]
    # Reverse the one-hot encoding and create a new column
    dfret["Payment Method"] = df1.idxmax(axis=1)
    dfret
    # Reverse Device Class
    df1 = dfret[["Low Device", "Mid Device", "High Device"]]
    # Reverse the one-hot encoding and create a new column
    dfret["Device Class"] = df1.idxmax(axis=1)
    dfret['Device Class'].replace(["Low Device", "Mid Device", "High Device"],["Low End", "Mid End", "High End"], inplace=True)
    # Reverse Products
    dfret['Games Product'].replace([0,1],['No','Yes'], inplace=True)
    dfret['Video Product'].replace([0,1],['No','Yes'], inplace=True)
    dfret['Music Product'].replace([0,1],['No','Yes'], inplace=True)
    dfret['Education Product'].replace([0,1],['No','Yes'], inplace=True)
    dfret['Use MyApp'].replace([0,1],['No','Yes'], inplace=True)
    # Reverse internet service
    dfret['Games Product'] = np.where(dfret['Internet Service'] == 0, 'No internet service' , dfret['Games Product'])
    dfret['Video Product'] = np.where(dfret['Internet Service'] == 0, 'No internet service' , dfret['Video Product'])
    dfret['Music Product'] = np.where(dfret['Internet Service'] == 0, 'No internet service' , dfret['Music Product'])
    dfret['Education Product'] = np.where(dfret['Internet Service'] == 0, 'No internet service' , dfret['Education Product'])
    dfret['Use MyApp'] = np.where(dfret['Internet Service'] == 0, 'No internet service' , dfret['Use MyApp'])
    # Reverse others
    dfret['Call Center'].replace([0,1],['No','Yes'], inplace=True)
    dfret['Location'].replace([0,1],['Jakarta','Bandung'], inplace=True)
    dfret['Churn Label'].replace([1,0],['Yes','No'], inplace=True)
    dfret = dfret.drop(["Debit", "Credit", "Pulsa", "Digital Wallet","Low Device", "Mid Device", "High Device","Internet Service"],axis=1)
    return dfret

In [7]:
def encode(df):
    df['Low Device'] = np.where(df['Device Class'] == 'Low End', 1, 0)
    df['Mid Device'] = np.where(df['Device Class'] == 'Mid End', 1, 0)
    df['High Device'] = np.where(df['Device Class'] == 'High End', 1, 0)
    df = df.drop('Device Class', axis = 1)
    df['Internet Service'] = np.where(df['Games Product'] == 'No internet service', 0 , 1)
    df['Games Product'].replace(['No','Yes','No internet service'], [0,1,0], inplace=True)
    df['Video Product'].replace(['No','Yes','No internet service'], [0,1,0], inplace=True)
    df['Music Product'].replace(['No','Yes','No internet service'], [0,1,0], inplace=True)
    df['Education Product'].replace(['No','Yes','No internet service'], [0,1,0], inplace=True)
    df['Use MyApp'].replace(['No','Yes','No internet service'], [0,1,0], inplace=True)
    df['Call Center'].replace(['No','Yes'], [0,1], inplace=True)
    df['Location'].replace(['Jakarta','Bandung'], [0,1], inplace=True)
    df['Churn Label'].replace(['Yes','No'], [1,0], inplace=True)
    df['Digital Wallet'] = np.where(df['Payment Method'] == 'Digital Wallet', 1,0)
    df['Pulsa'] = np.where(df['Payment Method'] == 'Pulsa', 1,0)
    df['Debit'] = np.where(df['Payment Method'] == 'Debit', 1,0)
    df['Credit'] = np.where(df['Payment Method'] == 'Credit', 1,0)
    df = df.drop('Payment Method', axis=1)
    return df

## Prediction
Masukan nilai pada ```x_months_to_be_predicted```
untuk melakukan prediksi ```x``` bulan kedepan

In [8]:
# MASUKAN DISINI
x_months_to_be_predicted = 0

In [9]:
df = pd.read_excel(filename,sheet_name='Sheet1')
df_encoded = encode(df)

In [10]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('churn_predictor.h5')
# Show the model architecture
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 18)               37        
 ion)                                                            
                                                                 
 dense_9 (Dense)             (None, 16)                304       
                                                                 
 dense_10 (Dense)            (None, 10)                170       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 522
Trainable params: 485
Non-trainable params: 37
_________________________________________________________________


In [12]:
def predict_churn(df,added_months):
        added = df['Tenure Months'] + added_months
        df['Tenure Months'] = added
        features = ['Tenure Months','Location','Low Device', 'Mid Device','High Device','Games Product', 'Music Product', 'Education Product', 'Video Product', 
                'Call Center', 'Use MyApp', 'Digital Wallet','Pulsa','Debit','Credit', 'Monthly Purchase (Thou. IDR)','CLTV (Predicted Thou. IDR)','Internet Service']
        X = df[features]
        y_pred = model.predict(X)
        y_final = []
        for pred in y_pred:
                if pred > 0.5:
                        y_final.append(1)
                else:
                        y_final.append(0)
        df['Churn Label'] = y_final
        return df        

In [15]:
prediction = predict_churn(df_encoded,x_months_to_be_predicted)

221/221 [==============================] - 1s 3ms/step


## Menyimpan Hasil

In [18]:
def save_output(df):
        df = decode(df)
        df.to_csv('predicted_months_'+str(x_months_to_be_predicted)+'.csv',index=False,sep=';')

In [19]:
save_output(prediction)